In [1]:
from init import *
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

autotime loaded.


In [2]:
class SummarizeText():
    def __init__(self, filename):
        self.filename = filename
        
    def read_text(self):
        with open(self.filename, 'r') as f:
            article = f.readlines()[0].split('.')
        sentences = [sentence.strip().replace("[^a-zA-Z]", " ").split(" ") for sentence in article]
        return sentences[:-1]
    
    def count_words(self, sent, all_words, stopwords):
        vector = [0] * len(all_words)
        for w in sent:
            if w not in stopwords:
                vector[all_words.index(w)] += 1
        return vector        
    
    def sentence_similarity(self, sent1, sent2, stopwords=None):
        if stopwords is None:
            stopwords = []
        sent1 = [w.lower() for w in sent1]
        sent2 = [w.lower() for w in sent2]
        all_words = list(set(sent1 + sent2))
        vector1 = self.count_words(sent1, all_words, stopwords)
        vector2 = self.count_words(sent2, all_words, stopwords)
        return 1 - cosine_distance(vector1, vector2)
    
    def similarity_matrix(self, sentences, stop_words):
        lng = len(sentences)
        idxs = range(lng)
        mat = np.zeros((lng, lng))
        for idx1 in idxs:
            for idx2 in idxs:
                if idx1 != idx2:
                    mat[idx1][idx2] = self.sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
        return mat
    
    def run(self, top_n=5):
        stop_words = stopwords.words('english')
        sentences = self.read_text()
        sim_mat = self.similarity_matrix(sentences, stop_words)
        sim_graph = nx.from_numpy_array(sim_mat)
        scores = nx.pagerank(sim_graph)        
        ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)  
        summary = [' '.join(ranked_sentence[i][1]) for i in range(top_n)]
        summary = '. '.join(summary) + '.'
        print(summary)

time: 13.5 ms


In [3]:
summ = SummarizeText('trump.txt')
summ.run(top_n=4)

President Trump made the decision to pull the troops - about half the number the United States has in Afghanistan now - at the same time he decided to pull American forces out of Syria, one official said. WASHINGTON - The Trump administration has ordered the military to start withdrawing roughly 7,000 troops from Afghanistan in the coming months, two defense officials said Thursday, an abrupt shift in the 17-year-old war there and a decision that stunned Afghan officials, who said they had not been briefed on the plans. Though Pentagon officials have said the influx of forces - coupled with a more aggressive air campaign - was helping the war effort, Afghan forces continued to take nearly unsustainable levels of casualties and lose ground to the Taliban. Mattis, he begrudgingly pledged an additional 4,000 troops to the Afghan campaign to try to hasten an end to the conflict.
time: 204 ms
